In [1]:
import pandas as pd
import numpy as np
import os
import random
import tensorflow as tf
import cv2
from tqdm import tqdm
import datetime
from tensorflow import keras
from tensorflow.keras.layers import Conv2D, MaxPooling2D, UpSampling2D, Concatenate 
from tensorflow.keras.layers import Input, Add, Conv2DTranspose
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.applications import VGG16
from tensorflow.keras.optimizers import SGD, Adam
from tensorflow.keras.losses import SparseCategoricalCrossentropy, MeanSquaredError, BinaryCrossentropy
from tensorflow.keras.utils import plot_model
from tensorflow.keras import callbacks

from  matplotlib import pyplot as plt
import matplotlib.image as mpimg
from IPython.display import clear_output
%matplotlib inline

from IPython.display import HTML
from base64 import b64encode

import os
from PIL import Image

steps:
- read images from paths
- build architecture of unet (https://www.kaggle.com/code/utkarshsaxenadn/breast-cancer-image-segmentation-attention-unet#Encoder)
- compile model, summarize, make sure doesn't exceed 2m params
- loss = meaniou
- predict

# |Read

In [1]:
test_img_path = '../Fire_Dataset/test/image/'+'fire/'
test_mask_path = '../Fire_Dataset/test/mask/' + 'fire/'
train_img_path = '../Fire_Dataset/train/image/' + 'fire/'
train_mask_path = '../Fire_Dataset/train/mask/'+ 'fire/'
validation_img_path = '../Fire_Dataset/validation/image/'+ 'fire/'
validation_mask_path = '../Fire_Dataset/validation/mask/'+ 'fire/'

In [6]:
test_img_path = '../Fire_Dataset/test/image/'+''
test_mask_path = '../Fire_Dataset/test/mask/' + ''
train_img_path = '../Fire_Dataset/train/image/' + ''
train_mask_path = '../Fire_Dataset/train/mask/'+ ''
validation_img_path = '../Fire_Dataset/validation/image/'+ ''
validation_mask_path = '../Fire_Dataset/validation/mask/'+ ''

In [7]:
def read_image(folder_path):
   
    def load_image(filename, shape):
        image = tf.keras.preprocessing.image.load_img(filename, target_size=(128,128))
        image = tf.keras.preprocessing.image.img_to_array(image)
        # image = cv2.imread(filename)
        # image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        return image / 255.0  # normalize pixel values to range [0, 1]

    images = np.array([load_image(os.path.join(folder_path, filename), (128,128)) for filename in os.listdir(folder_path)])
            
    return images

In [8]:
# with tf.device('/GPU:0'):
#     train_images = read_image(train_img_path)
#     train_masks = read_image(train_mask_path)
#     validation_images = read_image(validation_img_path)
#     validation_masks = read_image(validation_mask_path)

#     test_images = read_image(test_img_path)
#     test_masks = read_image(test_mask_path)

In [9]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

def get_datagen(img_path, mask_path):
        seed = 909 # (IMPORTANT) to transform image and corresponding mask with same augmentation parameter.
        image_datagen = ImageDataGenerator(width_shift_range=0.1,
                        height_shift_range=0.1,) # custom fuction for each image you can use resnet one too.

        mask_datagen = ImageDataGenerator(width_shift_range=0.1,
                        height_shift_range=0.1,)  # to make mask as feedable formate (256,256,1)


        train_img =image_datagen.flow_from_directory(img_path,
                                                            class_mode=None, seed=seed)

        train_mask = mask_datagen.flow_from_directory(mask_path,
                                                        class_mode=None, seed=seed)

        return zip(train_img, train_mask)

In [10]:
train =get_datagen(train_img_path, train_mask_path)
validation =get_datagen(validation_img_path, validation_mask_path)
test =get_datagen(test_img_path, test_mask_path)



Found 24925 images belonging to 1 classes.
Found 24925 images belonging to 1 classes.
Found 8307 images belonging to 1 classes.
Found 8307 images belonging to 1 classes.
Found 8304 images belonging to 1 classes.
Found 8304 images belonging to 1 classes.


# Constructing Unet

In [5]:
# common
import os
import keras
import numpy as np
import pandas as pd
from glob import glob
import tensorflow as tf

# Data
from tensorflow.keras.utils import to_categorical

# Data Viz
import matplotlib.pyplot as plt

# Model 
from keras.models import Model
from keras.layers import Layer
from keras.layers import Conv2D
from keras.layers import Dropout
from keras.layers import UpSampling2D
from keras.layers import concatenate
from keras.layers import Add
from keras.layers import Multiply
from keras.layers import Input
from keras.layers import MaxPool2D
from keras.layers import BatchNormalization

# Callbacks 
from keras.callbacks import Callback
from keras.callbacks import EarlyStopping
from keras.callbacks import ModelCheckpoint

# Metrics
from keras.metrics import MeanIoU

In [10]:
#Build the model
inputs = tf.keras.layers.Input((128, 128, 3))
s = tf.keras.layers.Lambda(lambda x: x / 255)(inputs)

#Contraction path
c1 = tf.keras.layers.Conv2D(16, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(s)
c1 = tf.keras.layers.Dropout(0.1)(c1)
c1 = tf.keras.layers.Conv2D(16, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c1)
p1 = tf.keras.layers.MaxPooling2D((2, 2))(c1)

c2 = tf.keras.layers.Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(p1)
c2 = tf.keras.layers.Dropout(0.1)(c2)
c2 = tf.keras.layers.Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c2)
p2 = tf.keras.layers.MaxPooling2D((2, 2))(c2)
 
c3 = tf.keras.layers.Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(p2)
c3 = tf.keras.layers.Dropout(0.2)(c3)
c3 = tf.keras.layers.Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c3)
p3 = tf.keras.layers.MaxPooling2D((2, 2))(c3)
 
c4 = tf.keras.layers.Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(p3)
c4 = tf.keras.layers.Dropout(0.2)(c4)
c4 = tf.keras.layers.Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c4)
p4 = tf.keras.layers.MaxPooling2D(pool_size=(2, 2))(c4)
 
c5 = tf.keras.layers.Conv2D(256, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(p4)
c5 = tf.keras.layers.Dropout(0.3)(c5)
c5 = tf.keras.layers.Conv2D(256, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c5)

#Expansive path 
u6 = tf.keras.layers.Conv2DTranspose(128, (2, 2), strides=(2, 2), padding='same')(c5)
u6 = tf.keras.layers.concatenate([u6, c4])
c6 = tf.keras.layers.Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(u6)
c6 = tf.keras.layers.Dropout(0.2)(c6)
c6 = tf.keras.layers.Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c6)
 
u7 = tf.keras.layers.Conv2DTranspose(64, (2, 2), strides=(2, 2), padding='same')(c6)
u7 = tf.keras.layers.concatenate([u7, c3])
c7 = tf.keras.layers.Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(u7)
c7 = tf.keras.layers.Dropout(0.2)(c7)
c7 = tf.keras.layers.Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c7)
 
u8 = tf.keras.layers.Conv2DTranspose(32, (2, 2), strides=(2, 2), padding='same')(c7)
u8 = tf.keras.layers.concatenate([u8, c2])
c8 = tf.keras.layers.Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(u8)
c8 = tf.keras.layers.Dropout(0.1)(c8)
c8 = tf.keras.layers.Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c8)
 
u9 = tf.keras.layers.Conv2DTranspose(16, (2, 2), strides=(2, 2), padding='same')(c8)
u9 = tf.keras.layers.concatenate([u9, c1], axis=3)
c9 = tf.keras.layers.Conv2D(16, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(u9)
c9 = tf.keras.layers.Dropout(0.1)(c9)
c9 = tf.keras.layers.Conv2D(16, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c9)
 
outputs = tf.keras.layers.Conv2D(1, (1, 1), activation='sigmoid')(c9)
 
model = tf.keras.Model(inputs=[inputs], outputs=[outputs])
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model.summary()

################################
#Modelcheckpoint
checkpointer = tf.keras.callbacks.ModelCheckpoint('model_for_nuclei.h5', verbose=1, save_best_only=True)

callbacks = [
        tf.keras.callbacks.EarlyStopping(patience=2, monitor='val_loss'),
        tf.keras.callbacks.TensorBoard(log_dir='logs')]

results = model.fit(train_images, train_masks, validation_data = (validation_images, validation_masks), batch_size=16, epochs=25, callbacks=callbacks)

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 128, 128, 3  0           []                               
                                )]                                                                
                                                                                                  
 lambda (Lambda)                (None, 128, 128, 3)  0           ['input_1[0][0]']                
                                                                                                  
 conv2d (Conv2D)                (None, 128, 128, 16  448         ['lambda[0][0]']                 
                                )                                                                 
                                                                                              

InternalError: Failed copying input tensor from /job:localhost/replica:0/task:0/device:CPU:0 to /job:localhost/replica:0/task:0/device:GPU:0 in order to run _EagerConst: Dst tensor is not initialized.

# Training

In [ ]:
# Config Training
BATCH_SIZE = 8
SPE = 24925 //BATCH_SIZE

# Training
results = model.fit_generator(
    train,
    validation_data=validation,
    epochs=20, # 15 will be enough for a good Model for better model go with 20+
    steps_per_epoch=SPE,
    callbacks=cb
)

C:\Users\Sistem Cerdas Two\AppData\Local\Temp\ipykernel_7012\821797379.py:6: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  results = model.fit_generator(


Epoch 1/20


InvalidArgumentError: Graph execution error:

Detected at node 'model/Attention1/multiply/mul' defined at (most recent call last):
    File "c:\Users\Sistem Cerdas Two\anaconda3\lib\runpy.py", line 197, in _run_module_as_main
      return _run_code(code, main_globals, None,
    File "c:\Users\Sistem Cerdas Two\anaconda3\lib\runpy.py", line 87, in _run_code
      exec(code, run_globals)
    File "c:\Users\Sistem Cerdas Two\anaconda3\lib\site-packages\ipykernel_launcher.py", line 16, in <module>
      app.launch_new_instance()
    File "c:\Users\Sistem Cerdas Two\anaconda3\lib\site-packages\traitlets\config\application.py", line 846, in launch_instance
      app.start()
    File "c:\Users\Sistem Cerdas Two\anaconda3\lib\site-packages\ipykernel\kernelapp.py", line 677, in start
      self.io_loop.start()
    File "c:\Users\Sistem Cerdas Two\anaconda3\lib\site-packages\tornado\platform\asyncio.py", line 199, in start
      self.asyncio_loop.run_forever()
    File "c:\Users\Sistem Cerdas Tw

In [ ]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 128, 128, 3  0           []                               
                                )]                                                                
                                                                                                  
 Encoder1 (EncoderBlock)        ((None, 64, 64, 32)  10144       ['input_1[0][0]']                
                                , (None, 128, 128,                                                
                                32))                                                              
                                                                                                  
 Encoder2 (EncoderBlock)        ((None, 32, 32, 64)  55424       ['Encoder1[0][0]']           